# Fluid Flow

## Parabolic Slider

We want to investigate the pressure and flow induced by a 2D parabolic slider using non-periodic boundary conditions.

Features of this use case:
- $U = 50$ m/s, $V = 0$ m/s
- $h_{\mathrm{min}} = 1 \cdot 10^{-5}$ m, $h_{\mathrm{max}} = 6.6 \cdot 10^{-5}$ m
- BC: $p_{\mathrm{BC}} = 1$ bar, $\frac{\partial j_x}{x}_{\mathrm{BC}} = 0$, $\frac{\partial j_y}{y}_{\mathrm{BC}} = 0$

$\rightarrow$ **[YAML File](./examples/parabolic_slider_2d.yaml)**

Load the problem configuration and observe the height profile:

In [ ]:
from GaPFlow.problem import Problem

problem = Problem.from_yaml("examples/parabolic_slider_2d.yaml")
problem.plot_topo()

Run the problem:

In [ ]:
problem.run()

And plot the solution:

In [ ]:
problem.plot()

## Couette Flow

We want to observe the stability of flow build-up by manually setting flow to zero after initializing the simulation.

$\rightarrow$ **[YAML File](./examples/couette_2d.yaml)**

In [ ]:
from GaPFlow.problem import Problem
import numpy as np

problem = Problem.from_yaml("examples/couette_2d.yaml")
problem.q[1] = 0. # Set jx = 0 everywhere

problem.run()

problem.animate()

## Lid-Driven Cavity

The lid-driven cavity is a classic CFD benchmark: a closed square box with stationary walls on 3 sides and a moving top wall (lid).

Features of this use case:
- Lid velocity $U = 0.5$ m/s at north boundary
- All other walls stationary (no-slip)
- Re = $\rho U L / \mu$ = 1 (low Reynolds number)
- Uses in-plane viscous diffusion terms (R23xy, R23yx) for true 2D flow

The moving lid drives a recirculating vortex pattern.

$\rightarrow$ **[YAML File](./examples/lid_driven_cavity.yaml)**

In [ ]:
from GaPFlow.problem import Problem
import numpy as np

problem = Problem.from_yaml("examples/lid_driven_cavity.yaml")

# BC callback for consistent wall velocity at lid
# Without this, constant jx with varying rho leads to varying wall velocity u = jx/rho
def jx_lid(ctx):
      """ctx is a BCContext with: problem, required_shape, ghost_slice, interior_slice, x_norm, y_norm"""
      u_wall = 1.0
      rho_ghost = ctx.problem.q[0][ctx.ghost_slice]
      return rho_ghost * u_wall

problem.set_bc_function('jx', 'N', jx_lid)

# Initialize with uniform density, zero velocity
problem.q[0][:] = 1.0
problem.q[1][:] = 0.0
problem.q[2][:] = 0.0

problem.run()

In [ ]:
import sys; sys.path.insert(0, '.')
from utils.plotting import plot_lid_driven_cavity  # type: ignore

fig = plot_lid_driven_cavity(problem)

# Energy

Fluid flow + activated energy equation

## Advection

Dynamic simulation of 2D temperature advection with a sine initial profile.

Setup:
- Periodic boundary conditions in x and y
- Wall motion U > 0 drives Couette flow which advects the temperature
- Initial temperature: half-sine profile in x, constant in y
- No thermal diffusion (k=0)

The temperature wave travels with the average flow velocity. For Couette flow with wall velocity U, the average velocity is U/2. With periodic boundaries, the sine profile wraps around the domain.

Since the setup is uniform in y (periodic BC, constant initial T in y), the 2D solution should match 1D behavior.

Terms used:
- `R31x`, `R31y`: Temperature convection (advection with velocity field)
- `R3T`: Energy time derivative
- Mass and momentum terms for density and velocity fields

$\rightarrow$ **[YAML File](./examples/temp_advection_2d.yaml)**

In [ ]:
# 2D Temperature advection
import numpy as np
from GaPFlow.problem import Problem

problem = Problem.from_yaml("examples/temp_advection_2d.yaml")

# Store temperature history for animation
T_history = {'time': [], 'T': [], 'T_2d': []}

def track_T():
    T_history['time'].append(problem.simtime)
    # Extract T along x at mid-y (for 1D animation)
    T_history['T'].append(problem.energy.temperature[:, problem.grid['Ny']//2 + 1].copy())
    # Store full 2D field (for 3D animation)
    T_history['T_2d'].append(problem.energy.temperature.copy())

problem.add_callback(track_T)
problem.run()

In [ ]:
# 1D animation of temperature advection
import sys; sys.path.insert(0, '.')

# Get grid parameters
L = problem.grid['Lx']
Nx = problem.grid['Nx']
dx = problem.grid['dx']
U = problem.geo['U']

# For Couette flow, average velocity is U/2
U_avg = U / 2

# Cell centers including ghost cells
x_vec = np.arange(Nx + 2) * dx - dx / 2
t_vec = np.array(T_history['time'])
T_numeric = np.array(T_history['T'])

In [ ]:
from utils.plotting import animate_advection  # type: ignore

animate_advection(x_vec, t_vec, T_numeric, L, U_avg)

In [ ]:
# 3D surface animation of temperature advection
from utils.plotting import animate_3d_surface  # type: ignore

# Get grid parameters
Lx = problem.grid['Lx']
Ly = problem.grid['Ly']
Nx = problem.grid['Nx']
Ny = problem.grid['Ny']
dx = problem.grid['dx']
dy = problem.grid['dy']

# Cell centers including ghost cells
x_vec = np.arange(Nx + 2) * dx - dx / 2
y_vec = np.arange(Ny + 2) * dy - dy / 2

# Use every nth timestep to reduce frames while showing full advection cycle
every_n = 2
t_vec = np.array(T_history['time'][::every_n])
T_2d = np.array(T_history['T_2d'][::every_n])

print(f"Animation: {len(t_vec)} frames, t = {t_vec[0]:.2f} to {t_vec[-1]:.2f} s")
print(f"One full period takes {Lx / (problem.geo['U'] / 2):.1f} s")

animate_3d_surface(x_vec, y_vec, t_vec, T_2d, Lx=Lx, Ly=Ly,
                   title='Temperature Advection', cmap='coolwarm', figsize=(7, 4.5))

# Deformation

Here we run again the parabolic slider use case but this time with elastic deformation activated

$\rightarrow$ **[YAML File](./examples/parabolic_slider_2d_deform.yaml)**

In [ ]:
from GaPFlow.problem import Problem

problem = Problem.from_yaml("examples/parabolic_slider_2d_deform.yaml")

In [ ]:
import numpy as np

def print_p_and_deform():
    p_field = problem.pressure.pressure
    p_max = p_field.max()
    u_field = problem.topo.deformation
    u_max = u_field.max()
    print(f"Max Pressure: {p_max:.2f} Pa, Max Deformation: {u_max:.6f} m")

problem.add_callback(print_p_and_deform)

In [ ]:
problem.run()

In [ ]:
problem.plot_topo()

In [ ]:
problem.plot()

Height profiles along `x` in the middle:

In [ ]:
h_field = problem.topo.h
h_slice = h_field[1:-1, h_field.shape[1]//2]

h_undef = problem.topo.h_undeformed
h_undef_slice = h_undef[1:-1, h_undef.shape[1]//2]

from matplotlib.pyplot import legend, plot
plot(h_slice)
plot(h_undef_slice)
legend(['Deformed Height', 'Undeformed Height'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Get fields (inner cells only, excluding ghost cells)
h_deformed = problem.topo.h[1:-1, 1:-1]
h_undeformed = problem.topo.h_undeformed[1:-1, 1:-1]

# Get coordinates
x = problem.topo.xx[1:-1, 1:-1]
y = problem.topo.yy[1:-1, 1:-1]

# Common colormap and z-axis limits for both plots
vmin = min(h_undeformed.min(), h_deformed.min())
vmax = max(h_undeformed.max(), h_deformed.max())
zmin, zmax = vmin * 1e6, vmax * 1e6

# Create figure with two 3D subplots
fig = plt.figure(figsize=(14, 5), layout='constrained')

# Undeformed state
ax1 = fig.add_subplot(121, projection='3d')
surf1 = ax1.plot_surface(x * 1e3, y * 1e3, h_undeformed * 1e6, 
                          cmap='viridis_r', vmin=zmin, vmax=zmax,
                          edgecolor='none', alpha=0.9)
ax1.set_xlabel('x [mm]')
ax1.set_ylabel('y [mm]')
ax1.set_zlabel('h [µm]')
ax1.set_zlim(zmin, zmax)
ax1.set_title('Undeformed')
ax1.invert_zaxis()
ax1.view_init(elev=25, azim=-60)

# Deformed state
ax2 = fig.add_subplot(122, projection='3d')
surf2 = ax2.plot_surface(x * 1e3, y * 1e3, h_deformed * 1e6,
                          cmap='viridis_r', vmin=zmin, vmax=zmax,
                          edgecolor='none', alpha=0.9)
ax2.set_xlabel('x [mm]')
ax2.set_ylabel('y [mm]')
ax2.set_zlabel('h [µm]')
ax2.set_zlim(zmin, zmax)
ax2.set_title('Deformed')
ax2.invert_zaxis()
ax2.view_init(elev=25, azim=-60)

# Shared colorbar
fig.colorbar(surf2, ax=[ax1, ax2], shrink=0.6, label='h [µm]')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize

# Get fields (inner cells only, excluding ghost cells)
h_deformed = problem.topo.h[1:-1, 1:-1]
p_field = problem.pressure.pressure[1:-1, 1:-1]

# Get coordinates
x = problem.topo.xx[1:-1, 1:-1]
y = problem.topo.yy[1:-1, 1:-1]

# Create figure (same width as one subplot in the first figure)
fig = plt.figure(figsize=(8, 5), layout='constrained')
ax = fig.add_subplot(111, projection='3d')

# Normalize pressure for colormap
norm = Normalize(vmin=p_field.min(), vmax=p_field.max())
colors = cm.turbo(norm(p_field))

# Plot deformed surface colored by pressure
surf = ax.plot_surface(x * 1e3, y * 1e3, h_deformed * 1e6,
                        facecolors=colors, edgecolor='none', shade=False)

ax.set_xlabel('x [mm]')
ax.set_ylabel('y [mm]')
ax.set_zlabel('h [µm]')
ax.set_title('Deformed Surface (colored by pressure)')
ax.invert_zaxis()
ax.view_init(elev=25, azim=-60)

# Add colorbar for pressure with padding
mappable = cm.ScalarMappable(norm=norm, cmap='turbo')
fig.colorbar(mappable, ax=ax, shrink=0.6, pad=0.15, label='Pressure [Pa]')

plt.show()

# Deformation + Energy

$\rightarrow$ **[YAML File](./examples/parabolic_slider_2d_deform_energy.yaml)**

In [ ]:
from GaPFlow.problem import Problem

problem = Problem.from_yaml("examples/parabolic_slider_2d_deform_energy.yaml")

In [ ]:
problem.run()

In [ ]:
problem.plot()

In [ ]:
t_x = problem.energy.temperature[:, problem.grid['Ny']//2 + 1]
t_y = problem.energy.temperature[35, :]
import matplotlib.pyplot as plt
plt.plot(t_x)
plt.plot(t_y)
plt.xlabel('Position along slice')
plt.ylabel('Temperature [K]')
plt.legend(['T along x at mid-y', 'T along y at x=70 mm'])
plt.title('Temperature Distribution after Simulation')

In [ ]:
# 2D plot of the full temperature field
import matplotlib.pyplot as plt
import numpy as np

# Get temperature field (inner cells only, excluding ghost cells)
T_field = problem.energy.temperature[1:-1, 1:-1]

# Get coordinates
x = problem.topo.xx[1:-1, 1:-1]
y = problem.topo.yy[1:-1, 1:-1]

# Create 2D contour/heatmap plot
fig, ax = plt.subplots(figsize=(6, 4), layout='constrained')

# Use pcolormesh for the 2D temperature field
pcm = ax.pcolormesh(x * 1e3, y * 1e3, T_field, cmap='inferno', shading='auto')

ax.set_xlabel('x [mm]')
ax.set_ylabel('y [mm]')
ax.set_title('Temperature Field')
ax.set_aspect('equal')

# Add colorbar
cbar = fig.colorbar(pcm, ax=ax, label='Temperature [K]')

plt.show()

In [ ]:
# 2D plot of wall heat flux (top and bottom walls)
import matplotlib.pyplot as plt
from GaPFlow.models.heatflux import get_heatflux_2d

# Get heat flux fields
q_top, q_bot = get_heatflux_2d(problem)

# Get coordinates (inner cells only, excluding ghost cells)
x = problem.topo.xx[1:-1, 1:-1]
y = problem.topo.yy[1:-1, 1:-1]

# Create 2-panel plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5), layout='constrained')

# Top wall heat flux
pcm0 = axes[0].pcolormesh(x * 1e3, y * 1e3, q_top[1:-1, 1:-1], cmap='RdBu_r', shading='auto')
axes[0].set_xlabel('x [mm]')
axes[0].set_ylabel('y [mm]')
axes[0].set_title('Heat Flux at Top Wall')
axes[0].set_aspect('equal')
fig.colorbar(pcm0, ax=axes[0], label='q [W/m²]')

# Bottom wall heat flux
pcm1 = axes[1].pcolormesh(x * 1e3, y * 1e3, q_bot[1:-1, 1:-1], cmap='RdBu_r', shading='auto')
axes[1].set_xlabel('x [mm]')
axes[1].set_ylabel('y [mm]')
axes[1].set_title('Heat Flux at Bottom Wall')
axes[1].set_aspect('equal')
fig.colorbar(pcm1, ax=axes[1], label='q [W/m²]')

plt.show()

In [ ]:
# Comprehensive 2x3 solution overview
import matplotlib.pyplot as plt
import numpy as np
from GaPFlow.models.pressure import eos_pressure

# Get fields (inner cells only, excluding ghost cells)
rho = problem.q[0][1:-1, 1:-1].T  # Transpose for plotting (Ny, Nx)
jx = problem.q[1][1:-1, 1:-1].T
jy = problem.q[2][1:-1, 1:-1].T
h = problem.topo.h[1:-1, 1:-1]

# Grid parameters
Nx, Ny = problem.grid['Nx'], problem.grid['Ny']
dx, dy = problem.grid['dx'], problem.grid['dy']

# Grid coordinates (cell centers)
x = np.linspace(dx / 2, Nx * dx - dx / 2, Nx)
y = np.linspace(dy / 2, Ny * dy - dy / 2, Ny)
X, Y = np.meshgrid(x, y)

# Compute pressure from density
p = np.asarray(eos_pressure(rho, problem.prop))

# Create figure with subplots
fig, axes = plt.subplots(2, 3, figsize=(15, 9), layout='constrained')

# 1. Height field
ax1 = axes[0, 0]
c1 = ax1.contourf(X * 1000, Y * 1000, h.T * 1e6, levels=20, cmap='terrain')
fig.colorbar(c1, ax=ax1, label=r'$h$ [$\mu$m]')
ax1.set_xlabel('x [mm]')
ax1.set_ylabel('y [mm]')
ax1.set_title(r'Gap height $h$')
ax1.set_aspect('equal')

# 2. Density field with flux vectors
ax2 = axes[0, 1]
c2 = ax2.contourf(X * 1000, Y * 1000, rho, levels=20, cmap='viridis')
fig.colorbar(c2, ax=ax2, label=r'$\rho$ [kg/m³]')
# Quiver plot for flux
skip = max(1, Nx // 12)
jx_skip, jy_skip = jx[::skip, ::skip], jy[::skip, ::skip]
j_mag = np.sqrt(jx_skip**2 + jy_skip**2)
j_mag[j_mag == 0] = 1  # Avoid division by zero
jx_norm, jy_norm = jx_skip / j_mag, jy_skip / j_mag
ax2.quiver(X[::skip, ::skip] * 1000, Y[::skip, ::skip] * 1000,
           jx_norm, jy_norm,
           color='white', alpha=0.9, scale=15, width=0.004, headwidth=3, headlength=4)
ax2.set_xlabel('x [mm]')
ax2.set_ylabel('y [mm]')
ax2.set_title(r'Density $\rho$ with flux $\mathbf{j}$')
ax2.set_aspect('equal')

# 3. Pressure field
ax3 = axes[0, 2]
c3 = ax3.contourf(X * 1000, Y * 1000, p / 1000, levels=20, cmap='coolwarm')
fig.colorbar(c3, ax=ax3, label='p [kPa]')
ax3.set_xlabel('x [mm]')
ax3.set_ylabel('y [mm]')
ax3.set_title(f'Pressure field (EOS: {problem.prop["EOS"]})')
ax3.set_aspect('equal')

# 4. Mass flux jx field
ax4 = axes[1, 0]
c4 = ax4.contourf(X * 1000, Y * 1000, jx * 1000, levels=20, cmap='plasma')
fig.colorbar(c4, ax=ax4, label=r'$j_x \times 10^3$')
ax4.set_xlabel('x [mm]')
ax4.set_ylabel('y [mm]')
ax4.set_title(r'Mass flux $j_x$')
ax4.set_aspect('equal')

# 5. Mass flux jy field
ax5 = axes[1, 1]
c5 = ax5.contourf(X * 1000, Y * 1000, jy * 1000, levels=20, cmap='plasma')
fig.colorbar(c5, ax=ax5, label=r'$j_y \times 10^3$')
ax5.set_xlabel('x [mm]')
ax5.set_ylabel('y [mm]')
ax5.set_title(r'Mass flux $j_y$')
ax5.set_aspect('equal')

# 6. Centerline profiles
ax6 = axes[1, 2]
mid_y = rho.shape[0] // 2
mid_x = rho.shape[1] // 2
x_line = X[mid_y, :] * 1000
y_line = Y[:, mid_x] * 1000
p_x = p[mid_y, :] / 1000  # kPa along x at y=Ly/2
p_y = p[:, mid_x] / 1000  # kPa along y at x=Lx/2

ax6.plot(x_line, p_x, 'b-', linewidth=2, label=r'$p(x, L_y/2)$')
ax6.plot(y_line, p_y, 'r--', linewidth=2, label=r'$p(L_x/2, y)$')
ax6.set_xlabel('Position [mm]')
ax6.set_ylabel('p [kPa]')
ax6.set_title('Centerline pressure profiles')
ax6.legend()
ax6.grid(True, alpha=0.3)

fig.suptitle('2D FEM Solution Overview: Parabolic Slider with Deformation + Energy', fontsize=12)
plt.show()

In [ ]:
# 3D surface plot colored by temperature
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize

# Get fields (inner cells only, excluding ghost cells)
h_deformed = problem.topo.h[1:-1, 1:-1]
T_field = problem.energy.temperature[1:-1, 1:-1]

# Get coordinates
x = problem.topo.xx[1:-1, 1:-1]
y = problem.topo.yy[1:-1, 1:-1]

# Create figure
fig = plt.figure(figsize=(8, 5), layout='constrained')
ax = fig.add_subplot(111, projection='3d')

# Normalize temperature for colormap
norm = Normalize(vmin=T_field.min(), vmax=T_field.max())
colors = cm.inferno(norm(T_field))

# Plot deformed surface colored by temperature
surf = ax.plot_surface(x * 1e3, y * 1e3, h_deformed * 1e6,
                        facecolors=colors, edgecolor='none', shade=False)

ax.set_xlabel('x [mm]')
ax.set_ylabel('y [mm]')
ax.set_zlabel('h [µm]')
ax.set_title('Deformed Surface (colored by temperature)')
ax.invert_zaxis()
ax.view_init(elev=25, azim=-60)

# Add colorbar for temperature
mappable = cm.ScalarMappable(norm=norm, cmap='inferno')
fig.colorbar(mappable, ax=ax, shrink=0.6, pad=0.15, label='Temperature [K]')

plt.show()

In [ ]:
# Height profile along y at x=70mm
h_field = problem.topo.h
x_idx = int(0.070 / problem.grid['dx']) + 1  # +1 for ghost cell offset
h_slice = h_field[x_idx, 1:-1]

h_undef = problem.topo.h_undeformed
h_undef_slice = h_undef[x_idx, 1:-1]

from matplotlib.pyplot import legend, plot
plot(h_slice)
plot(h_undef_slice)
legend(['Deformed Height', 'Undeformed Height'])